In [21]:
%ls

ControlLoRA/  LoRA/  retraining-free-pruning/  TextPruner/


In [23]:
import dask.dataframe as dd
import tensorflow as tf

# tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

# from waymo_open_dataset import v2

AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [8]:
# table_path = f'{dataset_dir}/camera_image/{context_name}.parquet'
table_path = "/scratch/network/xs6153/ControlLoRA/data/waymo/parquet/training_camera_image_10072140764565668044_4060_000_4080_000.parquet"
print(f'Reading a single shard from a single component {table_path}')
table = dd.read_parquet(table_path)
table.head()
     

Reading a single shard from a single component /scratch/network/xs6153/ControlLoRA/data/waymo/parquet/training_camera_image_10072140764565668044_4060_000_4080_000.parquet


,key.segment_context_name,key.frame_timestamp_micros,key.camera_name,[CameraImageComponent].image,[CameraImageComponent].pose.transform,[CameraImageComponent].velocity.linear_velocity.x,[CameraImageComponent].velocity.linear_velocity.y,[CameraImageComponent].velocity.linear_velocity.z,[CameraImageComponent].velocity.angular_velocity.x,[CameraImageComponent].velocity.angular_velocity.y,[CameraImageComponent].velocity.angular_velocity.z,[CameraImageComponent].pose_timestamp,[CameraImageComponent].rolling_shutter_params.shutter,[CameraImageComponent].rolling_shutter_params.camera_trigger_time,[CameraImageComponent].rolling_shutter_params.camera_readout_done_time
index,,,,,,,,,,,,,,,
10072140764565668044_4060_000_4080_000;1552696050377412,10072140764565668044_4060_000_4080_000,1552696050377412,1,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[-0.15147063222705368, 0.9884559207446768, -0....",-0.619360,-4.164054,0.007329,-0.027379,0.001065,-0.106989,1.552696e+09,0.000593,1.552696e+09,1.552696e+09
10072140764565668044_4060_000_4080_000;1552696050377412,10072140764565668044_4060_000_4080_000,1552696050377412,2,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[-0.15042807234620018, 0.9886140992246901, -0....",-0.627232,-4.150014,0.010885,-0.022551,0.004087,-0.094383,1.552696e+09,0.000734,1.552696e+09,1.552696e+09
10072140764565668044_4060_000_4080_000;1552696050377412,10072140764565668044_4060_000_4080_000,1552696050377412,4,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[-0.14948023113664483, 0.9887567559736689, -0....",-0.607636,-4.140365,0.012506,-0.019300,0.005464,-0.100567,1.552696e+09,0.000593,1.552696e+09,1.552696e+09
10072140764565668044_4060_000_4080_000;1552696050377412,10072140764565668044_4060_000_4080_000,1552696050377412,3,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[-0.15251302363089506, 0.988296854519358, -0.0...",-0.642461,-4.174776,0.005497,-0.033009,-0.000892,-0.101826,1.552696e+09,0.000579,1.552696e+09,1.552696e+09
10072140764565668044_4060_000_4080_000;1552696050377412,10072140764565668044_4060_000_4080_000,1552696050377412,5,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"[-0.15459717859392152, 0.9879750055644265, -0....",-0.663348,-4.192964,0.004353,-0.040870,-0.002729,-0.107865,1.552696e+09,0.001451,1.552696e+09,1.552696e+09


In [16]:
def read(tag: str) -> dd.DataFrame:
  """Creates a Dask DataFrame for the component specified by its tag."""
  paths = "/scratch/network/xs6153/ControlLoRA/data/waymo/parquet/training_camera_image_10072140764565668044_4060_000_4080_000.parquet"
#   paths = tf.io.gfile.glob(f'{dataset_dir}/{tag}/{context_name}.parquet')
  return dd.read_parquet(paths)

In [17]:
# @title Basic Example (Camera images with labels)

# Lazily read camera images and boxes 
cam_image_df = read('camera_image')
cam_box_df = read('camera_box')

# Combine DataFrame for individual components into a single DataFrame.

# Camera cam_box_df will be grouped, so each row will have a camera image
# and all associated boxes.
image_w_box_df = v2.merge(cam_image_df, cam_box_df, right_group=True)

# Show raw data
image_w_box_df.head()

# Example how to access data fields via v2 object-oriented API
print(f'Available {image_w_box_df.shape[0].compute()} rows:')
for i, (_, r) in enumerate(image_w_box_df.iterrows()):
  # Create component dataclasses for the raw data
  cam_image = v2.CameraImageComponent.from_dict(r)
  cam_box = v2.CameraBoxComponent.from_dict(r)
  print(
      f'context_name: {cam_image.key.segment_context_name}'
      f' ts: {cam_image.key.frame_timestamp_micros}'
      f' camera_name: {cam_image.key.camera_name}'
      f' image size: {len(cam_image.image)} bytes.'
      f' Has {len(cam_box.key.camera_object_id)} camera labels:'
  )

  for j, (object_id, x, y) in enumerate(zip(
      cam_box.key.camera_object_id, cam_box.box.center.x, cam_box.box.center.y
  )):
    print(f'\tid: {object_id},  center: ({x:.1f}, {y:.1f}) px')
    if j > 2:
      print('\t...')
      break
  if i > 2:
    print('...')
    break

NameError: name 'v2' is not defined